In [3]:
import csv
import json
import sys
import time
import pandas as pd
from confluent_kafka import Producer
from dateutil.parser import parse
import socket

In [4]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg.value()), str(err)))
    else:
        print("Message produced: %s" % (str(msg.value())))

In [5]:
TOPIC_NAME = "review_stream"
KAFKA_SERVER = "kafka:9092" # Đã đổi
CSV_FILE_PATH = "test_data.csv" # Đảm bảo file này tồn tại
DELAY_SECONDS = 1

# Các cột nhãn gốc từ file CSV
ASPECT_COLUMNS = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']

print(f"Đang kết nối tới Kafka: {KAFKA_SERVER}")
print(f"Đang đọc từ file: {CSV_FILE_PATH}")
print(f"Đang gửi tới topic: {TOPIC_NAME}")

# Cấu hình Producer (dùng confluent_kafka)
conf = {'bootstrap.servers': KAFKA_SERVER,
        'client.id': socket.gethostname()}
producer = Producer(conf)

# Đọc file CSV bằng pandas
try:
    df = pd.read_csv(CSV_FILE_PATH)
    # Kiểm tra xem tất cả các cột cần thiết có tồn tại không
    required_columns = ['Review'] + ASPECT_COLUMNS
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"LỖI: Thiếu các cột sau trong {CSV_FILE_PATH}: {', '.join(missing_cols)}")
        sys.exit(1)
        
    print(f"Đã tải {len(df)} bản ghi từ file.")
    # Chuyển đổi NaN thành giá trị hợp lệ cho JSON (ví dụ: -99 hoặc None nếu Kafka xử lý được)
    # Hoặc đảm bảo dữ liệu gốc không có NaN trong các cột aspect
    # Ở đây ta giả định dữ liệu đã sạch hoặc fillna trước đó. Nếu không, cần thêm df.fillna(...)
    # Ví dụ: df[ASPECT_COLUMNS] = df[ASPECT_COLUMNS].fillna(-99) # Dùng -99 thay cho NaN


except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {CSV_FILE_PATH}")
    sys.exit(1)
except Exception as e:
    print(f"Lỗi khi đọc CSV: {e}")
    sys.exit(1)

Đang kết nối tới Kafka: localhost:9092
Đang đọc từ file: test_data.csv
Đang gửi tới topic: review_stream
Đã tải 2340 bản ghi từ file.


In [6]:
# Vòng lặp gửi dữ liệu
try:
    # Lặp qua từng hàng của DataFrame
    for index, row in df.iterrows():
        review_text = row['Review']
        if pd.isna(review_text):
            print("Bỏ qua bình luận rỗng (NaN)")
            continue
            
        # Tạo payload dạng JSON chứa cả review và các nhãn thật
        payload = {
            "review_text": str(review_text)
        }
        # Thêm các cột aspect vào payload
        for aspect in ASPECT_COLUMNS:
             # Đảm bảo giá trị là kiểu int hoặc float cơ bản, không phải numpy int/float
             label_value = row[aspect]
             # Chuyển đổi NaN sang None (hoặc -99 tùy cách xử lý) nếu chưa làm ở bước đọc file
             if pd.isna(label_value):
                 payload[aspect] = None # Hoặc -99
             else:
                 payload[aspect] = int(label_value) # Ép kiểu int cho chắc chắn


        # Chuyển đổi payload thành chuỗi JSON và encode sang utf-8
        message_value = json.dumps(payload).encode('utf-8')
        
        # Gửi tin nhắn (dùng confluent_kafka)
        producer.produce(TOPIC_NAME, value=message_value, callback=acked)
        
        # producer.poll(0) sẽ gọi các callback (acked) từ các lần produce() trước đó
        producer.poll(0)
        
        # Đợi 1 giây
        print("By 23521198 - Lê Nguyễn Hoàng Phúc")
        time.sleep(DELAY_SECONDS)

except KeyboardInterrupt:
    print("Đã dừng bởi người dùng.")
finally:
    # Đảm bảo tất cả tin nhắn trong hàng đợi đều được gửi
    print("Đang xả (flushing) hàng đợi producer...")
    producer.flush()
    print("Hoàn tất.")

By 23521360 - Phạm Nguyễn Thanh Sơn


Message produced: b'{"review_text": "Gi\\u00e0y h\\u01a1i c\\u00f3 m\\u00f9i n\\u1ed3ng, l\\u01b0u \\u00fd \\u0111\\u00f4i LA kh\\u00f4ng ph\\u1ea3i \\u0111\\u1ebf x\\u00e1m n\\u00ean mng c\\u00e2n nh\\u1eafc k\\u0129 nh\\u00e9 \\u1ea1 nhma v\\u1edbi gi\\u00e1 ti\\u1ec1n n\\u00e0y th\\u00ec l\\u00e0 oke", "Price": -1, "Shipping": -1, "Outlook": -1, "Quality": 2, "Size": -1, "Shop_Service": -1, "General": 2, "Others": -1}'
By 23521360 - Phạm Nguyễn Thanh Sơn
Message produced: b'{"review_text": "H\\u00e0ng v\\u1ec1 \\u0111\\u1eb9p l\\u1eafm nha ship th\\u00e2n thi\\u1ec7n \\u0111i gi\\u00e0y v\\u1eeba in\\u2764\\ufe0f", "Price": -1, "Shipping": 1, "Outlook": 1, "Quality": -1, "Size": -1, "Shop_Service": -1, "General": -1, "Others": -1}'
By 23521360 - Phạm Nguyễn Thanh Sơn
Message produced: b'{"review_text": "H\\u00e0ng \\u00f4k n\\u00ean mua D\\u00e0y r\\u1ea5t \\u0111\\u1eb9p", "Price": -1, "Shipping": -1, "Outlook": 1, "Quality": -1, "Size": -1, "Shop_Service": -1, "General": 2, "Other